### Imports

In [36]:
import json

import networkx as nx
import pandas as pd

from utils.utils_go import *

# %load_ext autotime

### Parameters

In [37]:
file = open("exp.json")
experiment = json.load(file)
exp_num = experiment["exp"]

file = open("output/{}/parameters.json".format(exp_num))
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

data_variations = params["data_variations"]
print("Data variations:", data_variations)

threshold_corr = params["threshold_corr"]
print("Threshold corr:\t", threshold_corr)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

Exp:		 exp11
Data variations: ['str']
Threshold corr:	 0.01
Groups id:	 ['HN', 'HV']
Subgroups id:	 {'HN': ['1', '2'], 'HV': ['1', '2']}


### Load dataset

In [38]:
# load dataset groups
df_join_raw = pd.read_csv("input/{}_raw.csv".format(exp), index_col=0)
df_join_raw = df_join_raw.iloc[:, 2:]
df_join_raw

,HN_1.1,HN_1.2,HN_1.3,HN_2.1,HN_2.2,HN_2.3,HV_1.1,HV_1.2,HV_1.3,HV_2.1,HV_2.3,HV_2.4,HV_2.5
0,173.67982,1.17128,1.17128,1.17128,1.17128,117.12768,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128,1.17128
1,241.33417,2.41334,2.41334,2.41334,2.41334,306.20944,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334,2.41334
2,3.46233,3.46233,3.46233,3.46233,3.46233,346.23291,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233,3.46233
3,117.64917,1.17649,1.17649,1.17649,1.17649,132.44109,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649,1.17649
4,144.98218,1.44982,1.44982,230.19943,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982,1.44982
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,1.96126,196.12572,1.96126,1.96126,1.96126
1858,1.00339,1.00339,1.00339,346.97250,100.33888,183.92720,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339,1.00339
1859,1.39055,1.39055,1.39055,1.39055,139.05484,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055,1.39055
1860,1.77838,606.15350,177.83754,703.60211,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838,1.77838


In [39]:
check_dataset(df_join_raw)

Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 24206


### Generate graphs

In [40]:
# logarithm
""" if has_transformation == "true":
    df_join_raw_log = log10_global(df_join_raw)
else:
    df_join_raw_log = df_join_raw.copy() """

df_join_raw_log = log10_global(df_join_raw)
df_join_raw_log.head()

,HN_1.1,HN_1.2,HN_1.3,HN_2.1,HN_2.2,HN_2.3,HV_1.1,HV_1.2,HV_1.3,HV_2.1,HV_2.3,HV_2.4,HV_2.5
0,2.239749,0.068661,0.068661,0.068661,0.068661,2.068660,0.068661,0.068661,0.068661,0.068661,0.068661,0.068661,0.068661
1,2.382619,0.382619,0.382619,0.382619,0.382619,2.486019,0.382619,0.382619,0.382619,0.382619,0.382619,0.382619,0.382619
2,0.539368,0.539368,0.539368,0.539368,0.539368,2.539368,0.539368,0.539368,0.539368,0.539368,0.539368,0.539368,0.539368
3,2.070589,0.070588,0.070588,0.070588,0.070588,2.122023,0.070588,0.070588,0.070588,0.070588,0.070588,0.070588,0.070588
4,2.161315,0.161314,0.161314,2.362104,0.161314,0.161314,0.161314,0.161314,0.161314,0.161314,0.161314,0.161314,0.161314


In [41]:
check_dataset(df_join_raw_log)

Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 24206


In [42]:
# split graph in groups and subgroups

list_df_groups_subgroups = split_groups_subgroups(df_join_raw_log, groups_id, subgroups_id)
list_df_groups_subgroups[0][0].head()

,HN_1.1,HN_1.2,HN_1.3
0,2.239749,0.068661,0.068661
1,2.382619,0.382619,0.382619
2,0.539368,0.539368,0.539368
3,2.070589,0.070588,0.070588
4,2.161315,0.161314,0.161314


In [43]:
check_dataset(list_df_groups_subgroups[0][0])

Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 5586


In [44]:
# transpose
list_groups_subgroups_t = transpose_global(list_df_groups_subgroups)
list_groups_subgroups_t[0][0]

,0,1,2,3,4,5,6,7,8,9,...,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861
0,2.239749,2.382619,0.539368,2.070589,2.161315,0.726236,0.84926,0.99214,1.349784,0.595302,...,0.105456,0.011587,0.261484,0.03081,2.153143,0.292535,0.00147,0.143187,0.250025,0.267481
1,0.068661,0.382619,0.539368,0.070588,0.161314,0.726236,0.84926,0.99214,1.349784,0.595302,...,0.105456,0.011587,0.261484,0.03081,2.150099,0.292535,0.00147,0.143187,2.782583,0.267481
2,0.068661,0.382619,0.539368,0.070588,0.161314,0.726236,0.84926,0.99214,1.349784,0.595302,...,0.105456,0.011587,0.261484,0.03081,0.025990,0.292535,0.00147,0.143187,2.250023,0.267481


In [45]:
check_dataset(list_groups_subgroups_t[0][0])

Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 5586


In [46]:
# correlation matrix

list_groups_subgroups_t_corr = correlation_global(exp, groups_id, subgroups_id, list_groups_subgroups_t, method="pearson", plot=True)
list_groups_subgroups_t_corr[0][0].head()

0it [00:00, ?it/s]/home/ealvarez/miniconda3/envs/metanet_3.10/lib/python3.10/site-packages/pingouin/correlation.py:954: RuntimeWarning: divide by zero encountered in divide
  D = np.diag(np.sqrt(1 / np.diag(Vi)))
/home/ealvarez/miniconda3/envs/metanet_3.10/lib/python3.10/site-packages/pingouin/correlation.py:955: RuntimeWarning: invalid value encountered in matmul
  pcor = -1 * (D @ Vi @ D)  # Partial correlation matrix
2it [00:02,  1.31s/it]
2it [00:05,  2.69s/it]
2it [00:08,  4.01s/it]


,0,1,2,3,4,5,6,7,8,9,...,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861
0,1.000000,-1.000000,-0.456772,-1.000000,-1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.40557,NaN,NaN,NaN,0.727504,NaN
1,-1.000000,1.000000,-0.456772,-1.000000,-1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.40557,NaN,NaN,NaN,0.727504,NaN
2,-0.456772,-0.456772,1.000000,-0.456772,-0.456772,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.99839,NaN,NaN,NaN,0.942650,NaN
3,-1.000000,-1.000000,-0.456772,1.000000,-1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.40557,NaN,NaN,NaN,0.727504,NaN
4,-1.000000,-1.000000,-0.456772,-1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.40557,NaN,NaN,NaN,0.727504,NaN


<Figure size 640x480 with 0 Axes>

In [47]:
check_dataset(list_groups_subgroups_t_corr[0][0])

Checking dataset
Count infinite:	 0
Count nan:	 2729876
Count negative:	 471328
Count zero:	 0
Count positive:	 265840


In [48]:
# build graph (corpus graphs)

# list_groups_subgroups_t_corr_g = build_graph_weight_global(exp, list_groups_subgroups_t_corr, groups_id, subgroups_id, threshold=0.5)
list_groups_subgroups_t_corr_g = build_graph_weight_global_directed(exp, list_groups_subgroups_t_corr, groups_id, subgroups_id, threshold=threshold_corr)
# list_groups_subgroups_t_corr_g = build_graph_weight_global_undirected(exp, list_groups_subgroups_t_corr, groups_id, subgroups_id, threshold=threshold_corr)
list_groups_subgroups_t_corr_g[0][0]

  0%|          | 0/2 [00:00<?, ?it/s]

HN 1 858 367131


 50%|█████     | 1/2 [00:01<00:01,  1.84s/it]

HN 2 1030 529274


HV 1 843 354903


100%|██████████| 2/2 [00:02<00:00,  1.44s/it]

HV 2 501 123154


,source,target,weight
0,0,1,-1.000000
1,0,2,-0.456772
2,0,3,-1.000000
3,0,4,-1.000000
4,0,10,-1.000000
...,...,...,...
367648,1841,1856,0.999487
367649,1841,1860,0.934104
367650,1842,1856,0.405570
367651,1842,1860,0.727504


In [49]:
# create dataset - nodes/edge data for DGL framework

for data_variation in data_variations:
    if data_variation == "none":
        create_graph_data_go_features_directed(exp, groups_id, subgroups_id, list_df_groups_subgroups)
        # create_graph_data_go_features_undirected(exp, groups_id, subgroups_id, list_df_groups_subgroups)
    else:
        # dynamic graph to static graph
        create_graph_data_other_go_features(exp, groups_id, subgroups_id, data_variation, list_df_groups_subgroups)

100%|██████████| 2/2 [00:01<00:00,  1.19it/s]
2it [00:00, 1051.47it/s]
100%|██████████| 2/2 [00:00<00:00,  2.23it/s]
2it [00:00, 1047.92it/s]
2it [00:07,  3.96s/it]


In [50]:
# details
list_details = []
    
for group in groups_id:
    subgroups = []
    for data_variation in data_variations:
        if data_variation == "none":
            subgroups += subgroups_id[group]
        else:
            subgroups += [data_variation]
    # print(subgroups)
    
    for subgroup in subgroups:
        df_edges = pd.read_csv("output/{}/preprocessing/graphs_data/edges_data_{}_{}.csv".format(exp, group, subgroup))

        G = nx.from_pandas_edgelist(df_edges.iloc[:, [0, 1]])
        list_details.append([group, subgroup, G.number_of_nodes(), G.number_of_edges(), nx.density(G)])

df_details = pd.DataFrame(list_details, columns=["Group", "Subgroup", "Num. nodes", "Num. edges", "Density"])
df_details.to_csv("output/{}/preprocessing/graphs_data/summary.csv".format(exp), index=False)

""" df_details = pd.read_csv("output/{}/preprocessing/graphs_data/summary.csv".format(exp))
df_details """

' df_details = pd.read_csv("output/{}/preprocessing/graphs_data/summary.csv".format(exp))\ndf_details '